In [ ]:
%matplotlib inline
from matplotlib import pyplot as plt
from PIL import Image
import numpy as np
import os 
import cv2
from tqdm import tqdm_notebook
from random import shuffle
import shutil
import pandas as pd

In [ ]:
import keras
from keras.preprocessing.image import ImageDataGenerator
from keras_tqdm import TQDMNotebookCallback
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Flatten
from keras.constraints import maxnorm
from keras.optimizers import Adam
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.utils import np_utils
from keras.callbacks import Callback

In [ ]:
def organize_datasets(path_to_data, n=4000, ratio=0.2):
    files = os.listdir(path_to_data)
    files = [os.path.join(path_to_data, f) for f in files]
    shuffle(files)
    files = files[:n]
    
    n = int(len(files) * ratio)
    val, train = files[:n], files[n:]
    

    shutil.rmtree('./data/')
    print('/Data/ removed')

    for c in ['Deity', 'Holy', 'Perception' 'Scenery']: 
        os.makedirs('./data/train/{0}/'.format(c))
        os.makedirs('./data/val/{0}/'.format(c))

    print('folders created !')

    for t in tqdm_notebook(train):
        if 'Deity' in t:
            shutil.copy2(t, os.path.join('.', 'data', 'train', 'Deity'))  
        elif 'Holy' in t:
            shutil.copy2(t, os.path.join('.', 'data', 'train', 'Holy'))
        elif 'Perception' in t:
            shutil.copy2(t, os.path.join('.', 'data', 'train', 'Perception')) 
        else:
            shutil.copy2(t, os.path.join('.', 'data', 'train', 'Scenery'))
     
    for v in tqdm_notebook(val):
        if 'Deity' in v:
            shutil.copy2(v, os.path.join('.', 'data', 'val', 'Deity'))
        if 'Holy' in v:
            shutil.copy2(v, os.path.join('.', 'data', 'val', 'Holy'))
        if 'Perception' in v:
            shutil.copy2(v, os.path.join('.', 'data', 'val', 'Perception'))
        else:
            shutil.copy2(v, os.path.join('.', 'data', 'val', 'Scenery'))
            
    print('Data copied!')

In [ ]:
batch_size = 32

train_datagen = ImageDataGenerator(rescale=1/255.,
                                    shear_range=0.2,
                                    zoom_range=0.2,
                                    horizontal_flip=True
                                    )
val_datagen = ImageDataGenerator(rescale=1/255.)

In [ ]:
train_generator = train_datagen.flow_from_directory(
        './data/train/',
        target_size=(224, 224),
        batch_size=batch_size,
        class_mode='categorical')

validation_generator = val_datagen.flow_from_directory(
        './data/val/',
        target_size=(224, 224),
        batch_size=batch_size,
        class_mode='categorical')


In [ ]:
model = Sequential()

model.add(Conv2D(64, kernel_size = (3,3), strides= (1,1), input_shape= (224,224,3), activation='relu', padding='same', name= 'conv_1'))
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))

model.add(Conv2D(128, (3, 3), activation='relu', padding='same', name= 'conv_2'))
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))

model.add(Conv2D(256, (3, 3), activation='relu', padding='same', name= 'conv_3'))
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))

model.add(Conv2D(256, (3, 3), activation='relu', padding='same', name= 'conv_4'))
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))

model.add(Conv2D(128, (3, 3), activation='relu', padding='same', name= 'conv_5'))
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
model.add(Dropout(0.5))

model.add(Flatten())
model.add(Dense(4096, activation='relu', name= 'fc_1'))
model.add(Dense(4096, activation='relu', name= 'fc_2'))
model.add(Dropout(0.5))
model.add(Dense(4, activation='softmax', name= 'fc_3'))

In [ ]:
model.summary()

In [ ]:
epochs = 50
adam = Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-8, decay=0.0)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
class LossHistory(Callback):
    def on_train_begin(self, logs={}):
        self.losses = []
        self.val_losses = []
        
    def on_epoch_end(self, batch, logs={}):
        self.losses.append(logs.get('loss'))
        self.val_losses.append(logs.get('val_loss'))
        
history = LossHistory()

## Callback for early stopping the training
early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss',
                              min_delta=0,
                              patience=2,
                              verbose=0, mode='auto')

In [ ]:
# Adam Specialization

# TensorFlow: learning_rate=0.001, beta1=0.9, beta2=0.999, epsilon=1e-08.
# Keras: lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0
# Torch: learning_rate=0.001, beta1=0.9, beta2=0.999, epsilon=1e-8

In [ ]:
n = 6600
ratio = 0.2

In [ ]:
fitted_model = model.fit_generator(
        train_generator,
        steps_per_epoch= int(n * (1-ratio)) // batch_size,
        epochs=15,
        validation_data=validation_generator,
        validation_steps= int(n * ratio) // batch_size,
        callbacks=[TQDMNotebookCallback(leave_issnner=True, leave_outer=True), early_stopping, history],
        verbose=0)